<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/10_4_Bidirectional_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.5 MB/s eta 0:00:00


## 10.4 Bidirectional Recurrent neural Networks

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### 10.4.1 Implementation from Scratch

In [4]:
class BiRNNScratch(d2l.Module):
  def __init__(self, num_inputs, num_hiddens, sigma=0.01):
    super().__init__()
    self.save_hyperparameters()

    self.f_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
    self.b_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
    self.num_hiddens *=2

In [5]:
@d2l.add_to_class(BiRNNScratch)
def forward(self, inputs, Hs=None):
  f_H, b_H = Hs if Hs is not None else (None, None)

  f_outputs, f_H = self.f_rnn(inputs, f_H)
  b_outputs, b_H = self.b_rnn(reversed(inputs), b_H)
  outputs = [torch.cat((f, b), -1) for f, b in zip(f_outputs, reversed(b_outputs))]

  return outputs, (f_H, b_H)

### 10.4.2 Concise Implementation

In [6]:
class BiGRU(d2l.RNN):
  def __init__(self, num_inputs, num_hiddens):
    d2l.Module.__init__(self)
    self.save_hyperparameters()

    self.rnn = nn.GRU(num_inputs, num_hiddens, bidirectional=True)
    self.num_hiddens = self.num_hiddens * 2